In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_rows',None)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

whenever we are going to start machine learning based project we need to collect the data first or import it from secondary source. belove is the list of stages which we are going to follow.
- Collect the Data / Import the data from source.
- Describe the data/dataset
- Perform Data Pre-processing
    - correct the data
    - create the data
    - complete the data
    - convert the data
- Exploratory Data Analysis
- Feature Engineering
- Feature Selection
- Create the Models
- Test/Validate the Models
- Evaluate the Model




# 1. Import the Data/Dataset

at the source there are three datasets available.
- training 
- testing
- submission sample data

In [ ]:
# importing the data from source
train_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

Selecting the 5 sample of the data for review the data of train/test.

In [ ]:
# print the sample of the data
train_data.sample(5)

In [ ]:
test_data.sample(5)

# 2. Describe the Data

extract the shape of the traininf and testing datasets.

In [ ]:
# shape of the dataset 
print("training have rows and columns:",train_data.shape)
print("test data have rows and columns:", test_data.shape)

testing data have greter than the training data.

In [ ]:
# info the data
train_data.info()

In [ ]:
test_data.info()

Describe the descriptive statistics of the data

In [ ]:
#describe the statistics in data
# train_data.describe(include='all')

In [ ]:
# test_data.describe(include='all')

check the null values for every columns

In [ ]:
# check data cleaning 4c correcting, completing, creating, 
# check the null values for every columns
# targets = train_data['SalePrice']
# train_data.drop(['SalePrice'], axis=1, inplace=True)
def check_null_status(train_data, test_data):
    null_data = pd.DataFrame({'train_null_data %':[None],'test_null_data %':[None]}, index=test_data.columns)
    for col in test_data.columns:
        null_data.loc[col] = (round(sum(train_data[col].isna())/train_data.shape[0],2)*100,round(sum(test_data[col].isna())/test_data.shape[0],2)*100)
    return null_data
check_null_status(train_data, test_data)

In [ ]:
# 

In [ ]:
# print(targets)

# 3. Data Pre-processing

In [ ]:
# remove the columns which have more than the 80% null values.
# train_data.dropna(thresh=103, axis=1, inplace=True)
# # test_data.dropna(how='any', thresh=105, axis=1, inplace=True)
train_data.drop(['Id','PoolQC','Fence','MiscFeature','Alley'], axis=1, inplace=True)
test_data.drop(['Id','PoolQC','Fence','MiscFeature','Alley'], axis=1, inplace=True)

In [ ]:
train_data.select_dtypes(include='O').columns

In [ ]:
test_data.select_dtypes(include='O').columns

In [ ]:
# after fixing null values problems 
check_null_status(train_data, test_data)

In [ ]:
train_data.describe()

In [ ]:
train_data.sample(10)

In [ ]:
import re
def create_new_feature(data):
    dt = re.findall(r'\d+\.*\d*', str(data))
    if dt:
        return float(dt[0])
    else:
        return 1
    
def create_new_cat_feature(data):
    dt = re.findall(r'[a-zA-Z]+', str(data))
    if dt:
        return str(dt[0])
    else:
        None
        
for col in ['LotShape','BldgType','HouseStyle']:
    train_data[str(col)+'_n'] = train_data[col].apply(create_new_feature)
    train_data[col] = train_data[col].apply(create_new_cat_feature)

    test_data[str(col)+'_n'] = test_data[col].apply(create_new_feature)
    test_data[col] = test_data[col].apply(create_new_cat_feature)

    # train_data[str(col)+'_n'] = train_data.HouseStyle.apply(create_new_feature)

Impute the missing data.

In [ ]:
sales = train_data['SalePrice']
train_data.drop(['SalePrice'], axis=1, inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer
## for numeric data
smplimput = SimpleImputer(strategy='mean')
train_data1 = pd.DataFrame(smplimput.fit_transform(train_data.select_dtypes(exclude='O')), columns=train_data.select_dtypes(exclude='O').columns)
test_data1 = pd.DataFrame(smplimput.transform(test_data.select_dtypes(exclude='O')), columns=test_data.select_dtypes(exclude='O').columns)
## for categorical data
smplimput2 = SimpleImputer(strategy='most_frequent')
train_data2 = pd.DataFrame(smplimput2.fit_transform(train_data.select_dtypes(include='O')), columns=train_data.select_dtypes(include='O').columns)
test_data2 = pd.DataFrame(smplimput2.transform(test_data.select_dtypes(include='O')), columns=test_data.select_dtypes(include='O').columns)

In [ ]:
train_data = pd.concat([train_data1,train_data2],axis=1, ignore_index=False)
train_data['SalePrice'] = sales
test_data = pd.concat([test_data1,test_data2],axis=1, ignore_index=False)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
check_null_status(train_data, test_data)

In [ ]:
test_data.sample(10)

In [ ]:
train_data.sample(10)

# 4. EDA

plot the data with valid and outlier data.
by using **WINSORIZATION METHOD(PERCENTILE CAPPING)**.

In [ ]:
# plot/detect the outlier data with the help of winsort.
def detect_outlier(data):
    outlier=[]
    q1 = np.percentile(data , 1)
    q3 = np.percentile(data , 99)
    for i in data:
        if i > q3 or i < q1:
            outlier.append(True)
        else:
            outlier.append(False)
    return outlier


In [ ]:
out_train = train_data.copy()
out_test = test_data.copy()


In [ ]:
out_train.sample(5)

In [ ]:
out_train.info()

In [ ]:
# train_data.dtypes

def plot_vars(data, features, target):
    fig, axis = plt.subplots(len(features), 2, figsize=(20,200))
    for i, col in enumerate(features):
        data['Outlier'] = detect_outlier(data[col])
        sns.scatterplot(data=data, x=col, y=target, ax=axis[i, 0], hue='Outlier')
        sns.kdeplot(data=data, x=col, ax=axis[i, 1], hue='Outlier', warn_singular=False)
            
    

**plot the training data.**

In [ ]:
colmns = out_train.select_dtypes(exclude=['O']).columns
# print(colmns)
plot_vars(out_train, colmns, 'SalePrice')

**plot the categorical data of training data**


In [ ]:
cat_colmns = train_data.select_dtypes(include='O').columns

def cate_plot(data, features):
    fig, axis = plt.subplots(len(features), 2, figsize=(20,200))
    for i, col in enumerate(features):
        sns.countplot(data=data, x=col, ax=axis[i, 0])
        plt.xticks(rotation=45)
        sns.histplot(data=data, x=col, ax=axis[i, 1], kde=True)
        plt.xticks(rotation=45)

cate_plot(out_train[cat_colmns].astype(str), cat_colmns)

**visualize the testing data**

In [ ]:
## numeric data
# colmns = out_test.select_dtypes(exclude=['O']).columns
# # print(colmns)
# plot_vars(out_test, colmns[:-1], colmns[-1])

In [ ]:
## testing data with categorical columns
# cat_colmns = out_test.select_dtypes(include=['O']).columns
# cate_plot(out_train[cat_colmns].astype(str), cat_colmns)

In [ ]:
# label Encode the categorical features in to numerical colums
from sklearn.preprocessing import LabelEncoder
lblencoder = LabelEncoder()


In [ ]:
trn_data = train_data.copy()
tst_data = test_data.copy()

In [ ]:
for col in trn_data.select_dtypes(include=['O']).columns:
    trn_data[col] = lblencoder.fit_transform(trn_data[col].astype(str))
    tst_data[col] = lblencoder.transform(tst_data[col].astype(str))

In [ ]:
targets = trn_data['SalePrice']
trn_data.drop(['SalePrice'], axis=1, inplace=True)
# normalize and standardized the data
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mnmx = MinMaxScaler()
stdnorm = StandardScaler()

##############
def standardized_normalized(data):
    df_col = data.columns
    data = mnmx.fit_transform(data)
    data = stdnorm.fit_transform(data)
    return pd.DataFrame(data, columns=df_col)

        

In [ ]:
trn_data.shape

In [ ]:
trn_data = standardized_normalized(trn_data)
tst_data = standardized_normalized(tst_data)

In [ ]:
trn_data.sample(10)

In [ ]:
#tests data
tst_data.sample(10)

we have done clean 4c's 
- correcting
- completing
- creating
- converting

Now we are going to create a model for house price prediction.


In [ ]:
# shoe the correlation
# plt.figure(figsize=(20,20))
# # sns.heatmap(trn_data.corr())
# sns.pairplot(trn_data)

In [ ]:
#create a Ml model for regresor.
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

In [ ]:
ml_algo = [
    LinearRegression(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    SVR(),
    XGBRegressor(),
    KNeighborsRegressor()
]

In [ ]:
# run the model and save the accuracy in the dataframe
ml_columns = ['MLAlgo Name', 'MLAlgo Parameters','MLAlgo Train MAE', 'MLAlgo Test MAE','MLAlgo Time']
ml_algo_compare = pd.DataFrame(columns = ml_columns)

#create table to compare MLA predictions
ml_algo_predict = pd.DataFrame(targets.copy().values, columns=['Actual Sales'])

In [ ]:
# targets
# ml_algo_predict

In [ ]:
# targets
# from sklearn.model_selection import train_test_split
# X_train,X_test, y_train, y_test = train_test_split(trn_data, targets, test_size=0.05, random_state=42)

In [ ]:
row_index=0
for alg in ml_algo:
    #set name and parameters
    MLA_name = alg.__class__.__name__
    ml_algo_compare.loc[row_index, 'MLAlgo Name'] = MLA_name
    ml_algo_compare.loc[row_index, 'MLAlgo Parameters'] = str(alg.get_params())
    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = cross_validate(alg, trn_data, targets, scoring= 'neg_mean_absolute_error',cv = 3, return_train_score=True)
#     print(cv_results)
    ml_algo_compare.loc[row_index, 'MLAlgo Time'] = cv_results['fit_time'].mean()
    ml_algo_compare.loc[row_index, 'MLAlgo Train MAE'] = cv_results['train_score'].mean()
    ml_algo_compare.loc[row_index, 'MLAlgo Test MAE'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets  
    #save MLA predictions - see section 6 for usage
    alg.fit(trn_data, targets)
    ml_algo_predict[MLA_name] = alg.predict(trn_data)
    row_index += 1
    

In [ ]:
# take a care on ml_algo_model
ml_algo_compare

In [ ]:
ml_algo_predict.sample(10)

In [ ]:
#  start from your decision tree model
dtmodel = RandomForestRegressor()
dtmodel.fit(trn_data, targets)


In [ ]:
#submission data
demo_sample = sample_sub.copy()

In [ ]:
# see the performance of model
from sklearn.metrics import mean_absolute_error, r2_score
trn_p = dtmodel.predict(trn_data)
tst_ = dtmodel.predict(tst_data)

In [ ]:
print("training mae:", mean_absolute_error(targets,trn_p))
# print("testing mae:", mean_absolute_error(y_test,tst_p))

print("training r2:", r2_score(targets,trn_p))
# print("testing r2:", r2_score(y_test,tst_p))

In [ ]:
demo_sample['SalePrice'] = tst_

In [ ]:
# demo_sample

In [ ]:
# save the file
demo_sample.to_csv("submission.csv", index=False)

In [ ]:
# thank u 
# next update will come soon!